In [1]:
# ********************Import Modules-1
import os
from my_rbf import RBF
from distances import EuclideanDistances
from my_utils import *
import torch
from CFNET import CFnet
from preprocess import *
# import torch.autograd
import torch.utils.data as data
import torch.optim as optim
import random
import sys
print("Pytorch version: ", torch.__version__)
torch.set_printoptions(threshold=10, sci_mode=False)
# def setup_seed(seed):
#      torch.manual_seed(seed)
#      torch.cuda.manual_seed_all(seed)
#      np.random.seed(seed)
#      random.seed(seed)
#      torch.backends.cudnn.deterministic = True
# setup_seed(1997)

Pytorch version:  1.8.1


In [2]:
# ********************Configure-2

# os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['OMP_NUM_THREADS']='1'
device = torch.device('cuda:'+str(1))
# print(device)
torch.cuda.set_device(device)
print("Current device: ", torch.cuda.get_device_name(0))

Current device:  GeForce RTX 3090


In [3]:
# ********************Initialize molecules' properties and indices-3

Z_np = np.array([14, 12, 12, 16, 12, 12, 16, 14, 12, 12, 16, 12, 16, 14, 12, 12, 16, 12, 12, 14, 16, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 16, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 14, 12, 14, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 14, 16, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 14, 16, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 14, 12, 14, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 16, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 14, 16, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 14, 12, 14, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 14, 16, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 1, 1, 1, 14, 12, 12, 16, 12, 12, 32, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 16, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 14, 12, 14, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 16, 16, 1, 1, 1, 1, 14, 12, 12, 16, 12, 32, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 32, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 14, 12, 14, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 14, 16, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 16, 16, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 32, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 32, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 14, 12, 14, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 1, 12, 12, 14, 12, 14, 12, 14, 12, 12, 16, 12, 12, 12, 16, 16, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 14, 12, 14, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 14, 12, 14, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 16, 16, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 12, 12, 14, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 14, 16, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 16, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 16, 16, 1, 1, 1, 1, 14, 12, 12, 16, 12, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 16, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 16, 16, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 16, 16, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 14, 16, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 16, 16, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 12, 12, 16, 12, 12, 12, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
batch_size = 1
nATOM = len(Z_np)  # the number of atoms
epochs = 1000
Z_ts = torch.tile(torch.tensor(Z_np.ravel(), dtype=torch.int64),(batch_size,))

# --------------model initialization
nFM = 64  # the number of feature maps
cutoff = 30.  # rbf, the range of distances
gap = 0.1  # rbf, the interval of centers
n_embeddings = 2000
n_interactions = 3
# ---------------

crystal_WT_test_cor = np.load("/home/caofan/RAF/crystal_WT/test/deepNN/cor_set_1.npy", allow_pickle=True)
Pot_array = np.load("/home/caofan/RAF/crystal_WT/test/deepNN/potential_1.npy", allow_pickle=True)

Pot_ts = torch.tensor(Pot_array)

# Assignment grad
R_ts = torch.tensor(crystal_WT_test_cor, requires_grad=True)

R_ts = R_ts.cuda() # out-of-place
E_ts = Pot_ts.cuda()
Z_ts = Z_ts.cuda()

train_size = int(len(Pot_array) * 0.8)
test_size = int(len(Pot_array) * 0.2)

data_pool = dataset(R_ts.float(), E_ts.float())
train_set, test_set = data.random_split(data_pool, [train_size, test_size])

train_loader = data.DataLoader(
        dataset=train_set,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,  # whether read data using MPI
    )

test_loader = data.DataLoader(
        dataset=test_set,
        batch_size=test_size,
        shuffle=True,
        num_workers=0,  # whether read data using MPI
    )

In [4]:
# ********************Initialize CFNET-4
cfnet = CFnet(n_interactions=n_interactions, nFM=nFM, cutoff=cutoff, gap=gap, n_embeddings=n_embeddings, gpu=True).cuda()
optimizer = optim.Adam(cfnet.parameters(), lr=0.00001)
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0.00001)
dir = "train_model_pro"
checkpoint_path = './{0}/'.format(dir)
if not os.path.exists(os.getcwd()+dir):
    os.system("mkdir ./{0}".format(dir))

def train():
    for epoch in range(epochs):
        for step, (R, E) in enumerate(train_loader):
            R = torch.squeeze(R)
            # keep grad
            R.retain_grad()
            mols = molecules(nATOM, Z_ts, R, batch_size)

            # idx_ik = seg_i
            idx_ik = mols["idx_ik"].cuda()  # [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]

            # idx_jk = idx_j  [1, 2, 3, 4, 0, 2, 3, 4, 0, 1, 3, 4, 0, 1, 2, 4, 0, 1, 2, 3]
            idx_jk = mols["idx_jk"].cuda()
            # print("idx_jk:", idx_jk, idx_cfnetjk.shape)
            offset = mols["offset"].cuda()  # [20, 3]  [[0.,0.,0.,],...,[0.,0.,0.,]]

            # [0,...19]
            seg_j = mols["seg_j"].cuda()  # 距离索引
            seg_i_sum = mols["seg_i_sum"].cuda()  # [ 0,  4,  8, 12, 16]
            seg_m = mols["seg_m"].cuda()

            ratio_j = torch.tensor(1., dtype=torch.float32).cuda()
            rho = torch.tensor(.01, dtype=torch.float32).cuda()
            cfnet.zero_grad()
            Ep = cfnet(z=Z_ts, r=R, offsets=offset, idx_ik=idx_ik, idx_jk=idx_jk, idx_j=idx_jk, seg_m=seg_m, seg_i=idx_ik,
                       seg_j=seg_j, ratio_j=ratio_j, seg_i_sum=seg_i_sum)
            # Ep = Ep.requires_grad_(True)
            Ep.backward(retain_graph=True)
            Fp = -R.grad
            optimizer.zero_grad()

            loss, errors = CalLoss(Ep, Fp, E, None, rho, fit_forces=False)

            loss.backward()

            optimizer.step()
            print(" ===============================\n",
                  "${:^6}|{:^21d}||\n".format("epochs", epoch),
                  "-------------------------------\n",
                  "${:^6}|{:^21d}||\n".format("step", step),
                  "-------------------------------\n",
                  "${:^6}|{:^21f}||\n".format("loss", loss),
                  # "-------------------------------\n",
                  # "${:^6}|{:^21f}||\n".format("acc", acc),
                  "===============================\n")

            if step % 5000 == 0:
                lowest_loss = loss
                torch.save({'epoch': epoch + 1,
                            'state_dict': cfnet.state_dict(),
                            'best_loss': lowest_loss,
                            'optimizer': optimizer.state_dict()},
                            checkpoint_path + 'm-' + str("%.4f" % lowest_loss) + '.pth.tar')
                torch.save(cfnet, './{0}/model_test.pt'.format(dir))

        lr_scheduler.step()  # update learning rate

In [5]:
train()

RuntimeError: CUDA out of memory. Tried to allocate 1.72 GiB (GPU 1; 23.70 GiB total capacity; 15.94 GiB already allocated; 568.69 MiB free; 16.24 GiB reserved in total by PyTorch)

In [16]:
cor_set = np.load("/home/caofan/RAF/crystal_WT/test/deepNN/cor_set.npy", allow_pickle=True)
potential = np.load("/home/caofan/RAF/crystal_WT/test/deepNN/potential.npy", allow_pickle=True)

In [17]:
num = 3125
for i in range(1, 41):
    cor = cor_set[num*(i-1):num*i]
    pot = potential[num*(i-1):num*i]
    np.save("/home/caofan/RAF/crystal_WT/test/deepNN/cor_set_{0}.npy".format(i), cor)
    np.save("/home/caofan/RAF/crystal_WT/test/deepNN/potential_{0}.npy".format(i), pot)

In [6]:
print("Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in cfnet.parameters())))

Total number of paramerters in networks is 130177  


In [9]:
total = sum([param.nelement() for param in cfnet.parameters()])
print(total)

130177
